## signals(features)

convert lines, words, x, y, w, h in html input tag into singals

In [27]:
def count_line(contents):
    if not contents: 
        return 0
    lines = contents.split('\n')
    return len(lines)

def mean_word(contents):
    if not contents: 
        return 0
    lines = contents.split('\n')
    words = [len(line.split(' ')) for line in lines]
    return np.mean(words)

def total_word(contents):
    if not contents:
        return 0
    lines = contents.split('\n')
    words = [len(line.split(' ')) for line in lines]
    return sum(words)

def std_w(data_w):
    if not data_w:
        return 0
    ws = map(int, data_w.split(','))
    return np.std(ws)

def mean_y(data_y):
    if not data_y:
        return 0
    ys = map(int, data_y.split(','))
    return np.mean([abs(ys[1:][i] - ys[:-1][i]) for i in range(0, len(ys[1:]))])

def mean_h(data_h):
    if not data_h:
        return 0
    hs = map(int, data_h.split(','))
    return np.mean(hs)

def mean_x(data_x):
    if not data_x:
        return 0
    xs = map(int, data_x.split(','))
    x_min = np.min(xs)
    return np.mean(map(lambda x:x-x_min, xs))

def get_page(page):
    return int(page.replace('Page', '').strip())

In [28]:
def convert_feature(df, target=1):
    df['count_line'] = df['content'].apply(count_line)
    df['mean_word'] = df['content'].apply(mean_word)
    df['total_word'] = df['content'].apply(total_word)
    df['std_w'] = df['data_w'].apply(std_w)
    df['mean_y'] = df['data_y'].apply(mean_y)
    df['mean_h'] = df['data_h'].apply(mean_h)
    df['mean_x'] = df['data_x'].apply(mean_x)
    df['page_num'] = df['page'].apply(get_page)
    df['target'] = target
    return df


## Read poem

In [29]:
import numpy as np
import pandas as pd

poems = pd.read_csv('../data/poem-austlit-20160412.csv')
convert_feature(poems, target=1)
print poems.shape

(500, 10)


,title,data_h,content,newspaper,data_w,date,data_y,data_x,article_id,page,count_line,mean_word,total_word,std_w,mean_y,mean_h,mean_x,page_num,target
0,Poets Corner,"144,308,78,30,42,36,36,39,37,40,38,36,38,38,39...",^^^i\ne^5\nLu ? r\nMAD POETS.\nf1' Some people...,The Central Queensland Herald,"349,576,451,249,669,713,474,588,476,489,390,65...",Thu 2 Jan 1936,"788,484,740,1048,1099,1139,1180,1232,1272,1310...","264,912,912,284,96,50,53,51,89,53,93,54,94,54,...",70348715,Page 8,48,5.729167,275,108.055636,59.340426,46.583333,87.979167,8,1
1,CORRESPONDENCE,"61,61,48,39,32,43,39,41,38,39,33,39,36,40,39,4...",CORRESPONDENCE\nVALE OPOSSUM^; :' ' ;':\n(To t...,The Central Queensland Herald,"505,454,370,679,439,679,498,679,255,677,411,67...",Thu 23 Jul 1931,"3185,3333,3455,3532,3573,3609,3652,3690,3730,3...","1195,1278,1294,1122,1197,1124,1160,1121,1196,1...",70280540,Page 47,45,4.711111,212,165.352278,49.795455,39.355556,76.600000,47,1
2,The Worshipper.,"55,37,40,32,34,30,30,35,35,35,28,30,34,30,32,2...",The Worshipper.\nI WOULD not nurse a lovely th...,The Brisbane Courier,"355,621,164,438,618,257,427,614,583,616,166,46...",Sat 4 Jan 1930,"5315,5462,5486,5523,5553,5584,5613,5670,5700,5...","535,396,396,430,397,492,427,398,429,399,492,42...",21502619,Page 20,45,4.844444,218,185.748929,37.045455,32.333333,43.200000,20,1
3,THE LOST MERCHANT.,"41,56,35,29,28,26,27,26,27,25,26,25,27,26,25,2...","TUB LOST MKRCHAXT. I\nOa "" HOPES THAT ARE BRIG...",Empire,"795,1006,927,896,890,519,605,561,602,522,570,5...",Sat 30 Jun 1860,"1243,1286,1343,1373,1397,1424,1448,1472,1495,1...","5000,4789,4869,4869,4871,4871,4873,4869,4870,4...",60412555,Page 4,60,7.816667,469,126.620969,25.152542,27.116667,93.200000,4,1
4,ENIGMA.—No. IV.,"32,38,36,37,38,32,38,38,37,35,38,39,41,37,33,3...",ENIGMA.— No. IV.\nI liave a dear partner for l...,Bathurst Free Press,"320,471,556,591,650,412,570,577,687,443,550,55...",Sat 21 Sep 1850,"3425,3491,3522,3554,3584,3653,3682,3714,3746,3...","633,423,458,425,458,420,450,424,458,422,457,41...",62215013,Page 6,20,6.350000,127,118.589744,42.157895,38.400000,100.950000,6,1
5,"""Pioneers.""","67,37,34,39,37,44,38,43,36,43,37,43,37,44,35,4...","""Pioneers.""\n(Written for the ""Town and Countr...",Australian Town and Country Journal,"444,931,707,1038,857,1041,393,1081,308,1038,18...",Sat 19 Dec 1896,"264,360,428,463,506,6033,6079,6113,6160,6191,6...","727,478,594,414,495,426,504,384,504,425,501,42...",71302175,Page 17,40,5.875000,235,336.377681,778.743590,39.775000,1180.225000,17,1
6,Original Poetry.,"56,61,50,55,62,41,41,47,47,43,49,41,42,85,43,4...",Original Poetry.\n[FOR THE SYDNEY GAZETTE.]\nS...,The Sydney Gazette and New South Wales Adverti...,"451,763,954,961,756,911,931,913,967,884,884,91...",Sat 24 Oct 1829,"276,345,407,444,489,618,657,693,729,773,805,84...","483,435,218,237,417,288,268,286,224,290,315,28...",2193690,Page 4,53,6.452830,342,181.358807,42.596154,45.018868,119.792453,4,1
7,Grandmother's Mission.,"51,47,47,43,50,48,47,48,48,49,46,51,47,50,49,4...","Grandmother's Mission.\n. Como, my darling, si...",Australian Town and Country Journal,"674,717,670,687,696,651,632,761,659,682,623,78...",Sat 13 Oct 1888,"1111,1192,1239,1285,1330,1377,1425,1471,1518,1...","2618,2498,2562,2517,2560,2515,2560,2514,2561,2...",71103152,Page 29,25,6.840000,171,81.807833,50.833333,49.600000,98.920000,29,1
8,Literary Matters.,"85,31,41,30,41,38,37,41,38,40,40,43,41,42,35,3...","Literary Matters.\nTHE CLOSE OF LIFE.\n""Evil i...",The Queenslander,"628,389,655,426,693,584,741,671,692,680,659,77...",Sat 12 Oct 1872,"773,934,992,1027,1082,1115,1172,1203,1238,1269...","5257,5364,5205,5236,5134,5101,5135,5101,5101,5...",27273046,Page 7,93,8.000000,744,95.970311,36.206522,38.731183,24.860215,7,1
9,THE CHILDREN'S CORNER.,"70,56,47,50,55,57,57,55,45,49,48,42,53,52,54,5...","THE CHILDREN'S CORNER,\nTo the Old Year.\nGood...",Australian Town and Country Journal,"967,462,437,373,751,802,680,827,637,460,440,69...",Sat 28 Dec 1889,"303,425,523,568,613

In [30]:
# others
others = pd.read_csv('../data/others-201604120925.csv')
convert_feature(others, target=0)
print others.shape

(493, 10)


C:\Users\uqbjeon\AppData\Local\Continuum\Miniconda2\envs\myenv\lib\site-packages\numpy\core\_methods.py:59: RuntimeWarning: Mean of empty slice.
  warnings.warn("Mean of empty slice.", RuntimeWarning)


,title,data_h,content,newspaper,data_w,date,data_y,data_x,article_id,page,count_line,mean_word,total_word,std_w,mean_y,mean_h,mean_x,page_num,target
0,THE FREETRADE CAUSE.,"23,49,28,27,32,32,28,26,31,28,29,27,30,27,28,3...",mi'. FREETRADE CAUSE.\nI TO THE EDITOR OF THE ...,The Sydney Morning Herald,"422,607,631,658,661,660,659,659,657,658,659,65...",Wed 11 Jul 1894,"5482,5529,5581,5609,5635,5661,5688,5716,5742,5...","4602,4468,4511,4484,4482,4482,4482,4481,4481,4...",13959399,Page 6,43,7.720930,332,136.393626,27.071429,28.255814,26.627907,6,0
1,BUSINESS AT THE CENTRAL POLICE COURT.,"36,43,37,39,40,39,40,40,39,35,41,41,41,40,36,2...",I BTJßlNESB AT THE CENTRAL POLICE I\nI COUKT.\...,The Sydney Morning Herald,"897,517,842,839,841,842,842,841,840,490,807,84...",Sat 14 Jun 1862,"399,429,470,503,536,569,604,638,672,708,738,77...","1125,1125,1151,1152,1151,1150,1150,1149,1150,1...",13230099,Page 7,141,7.914894,1116,177.283545,33.792857,36.120567,38.078014,7,0
2,INTERCOLONIAL SHIPPING.,"37,33,35,34,32,34,33,37,34,36,28,35,36,34,34,3...",INTERCOLONIAL SHIPPING.\nAunrvi o -Melbourne T...,The Mercury,"547,593,615,625,612,617,621,616,614,616,290,59...",Tue 21 Feb 1893,"1635,1676,1700,1725,1748,1772,1797,1819,1844,1...","1194,1173,1152,1151,1156,1152,1155,1152,1154,1...",13268199,Page 2,23,6.000000,138,67.330856,24.727273,33.956522,6.521739,2,0
3,NEWS OF THE DAY.,"30,19,38,31,35,34,36,33,36,32,36,36,37,37,38,3...",NEWS OF TEE BAY.\n?\nAccOTCDTxa to a cablegram...,The Sydney Morning Herald,"486,23,801,843,840,838,838,839,842,842,686,843...",Thu 15 Jul 1886,"4031,4079,4113,4152,4190,4228,4264,4302,4339,4...","474,709,339,297,298,300,301,298,297,296,297,33...",13643299,Page 7,668,8.785928,5869,106.463286,78.433283,36.699102,1659.856287,7,0
4,PARRAMATTA.,"31,43,30,34,34,33,34,40,29,35,37,38,37,38,34,3...",PARRAMATTA.\nFrom our correspondent.]\nMunicip...,The Sydney Morning Herald,"298,693,861,861,863,862,862,864,860,862,860,86...",Wed 5 Feb 1873,"630,690,740,771,803,834,865,896,927,958,988,10...","5049,4742,4769,4770,4767,4769,4769,4768,4770,4...",13319499,Page 3,43,8.534884,367,95.244004,32.166667,37.906977,33.069767,3,0
5,SYDNEY MARKETS.,"38,28,31,28,27,28,27,30,27,28,34,27,29,28,33,3...","SYDNEY MARKETS. ' ' . ;\nPoultry-Fowls, per co...",The Sydney Morning Herald,"690,727,708,741,714,719,705,729,726,737,677,71...",Sat 8 Apr 1871,"2372,2410,2433,2463,2489,2516,2543,2569,2595,2...","4990,4765,4788,4789,4790,4786,4788,4761,4763,4...",13210499,Page 3,79,6.430380,508,235.490893,37.500000,27.240506,235.113924,3,0
6,Advertising,"27,30,27,29,31,32,29,32",Vatoadib Dibcovebt Fon TUB Hair -If vouT hair ...,The Sydney Morning Herald,"662,691,688,688,686,89,667,639",Sat 25 Jul 1891,"6797,6821,6846,6866,6891,6914,6945,6967","3268,3242,3243,3243,3242,3245,3266,3243",13832499,Page 10,8,8.375000,67,194.334602,24.285714,29.625000,7.000000,10,0
7,Advertising,"33,25,27,28,31,28,30,32,24,31,30,26,29,25,28,3...",ABSTRACT OF SALES EY AUCT102T\n._-_ DAY.\nnjKv...,The Sydney Morning Herald,"624,183,459,679,630,510,671,632,575,682,635,41...",Thu 29 Jun 1893,"1329,1366,1433,1456,1479,1502,1524,1548,1579,1...","3838,4061,3913,3809,3858,3856,3810,3858,3857,3...",13935199,Page 9,76,6.868421,522,189.191396,24.266667,25.065789,41.776316,9,0
8,To the Editor of the Herald.,"34,37,39,37,37,37,39,37,36,39,34,37,34,35,34,3...","I To the M'ditor of the Herald. \\nSill,-Havin...",The Sydney Morning Herald,"899,847,844,847,848,845,847,847,846,870,847,84...",Wed 8 Oct 1862,"261,290,318,352,383,413,445,475,507,534,570,60...","3860,3888,3888,3886,3885,3886,3886,3886,3886,3...",13235199,Page 3,432,9.759259,4216,64.464105,47.078886,36.976852,473.812500,3,0
9,SCOTTISH AND AUSTRALIAN MINING COMPANY.,"34,31,30,38,40,38",SCOTTISH AND AUSTRALIAN MINING\nCOMPANY.\nLOND...,The Sydney Morning Herald,"767,207,351,730,770,563",Fri 26 Apr 1889,"2721,2771,2818,2856,2905,2956","2787,3066,3195,2825,2785,2781",13724399,Page 7,6,4.666667,28,217.568891,47.000000,35.166667,125.500000,7,0


In [42]:
all_df = pd.concat([poems, others])

In [43]:
all.shape

(993, 19)

In [44]:
from __future__ import division

import numpy as np
import operator

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.cross_validation import cross_val_score, KFold

from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

from sklearn import metrics
from sklearn.grid_search import GridSearchCV
from sklearn.pipeline import Pipeline

from sklearn.base import BaseEstimator
from sklearn.base import ClassifierMixin
from sklearn.preprocessing import LabelEncoder
from sklearn.externals import six
from sklearn.base import clone
from sklearn.pipeline import _name_estimators


In [169]:
class MajorityVoteClassifier(object):
    def __init__(self):
        self.classifiers = []
        self.weights = None

    def read_files(self):
        poems = pd.read_csv('../data/poem-austlit-20160412.csv')
        convert_feature(poems, target=1)
        others = pd.read_csv('../data/poem-austlit-20160412.csv')
        convert_feature(others, target=0)
        
    def process_data(self, df):
        format_features = ['count_line', 'mean_word', 'total_word', 'std_w', 'mean_y', 'mean_h', 'mean_x', 'page_num']
        content_features = 'content'
        y_feature = ['target']
        
        X_format = df[format_features].values
        X_format[X_format > 1000] = 1000
        X_format[np.isnan(X_format)] = 0
        self.X_format = X_format
        
        X_content = df[content_features].values
        X_content = CountVectorizer(max_df=10, min_df=1).fit_transform(X_content)
        X_content = TfidfTransformer().fit_transform(X_content)
        self.X_content = X_content
        
        y = df[y_feature].values
        y = y.flatten()
        self.y = y
        
    def predict(self):
        row_count, _ = self.X_format.shape
        test_count = 100
        idx = np.random.randint(0, row_count, test_count)
        X_format = self.X_format[idx]
        X_content = self.X_content[idx]
        print(X_format)
        print(X_content)
        print self.y
        print(idx)
        y = self.y[idx]
        print(y)
        
        predictions = [self._predict(self.X_format[i, :], self.X_content[i, :])  for i in range(0, test_count)]
        predictions = np.array(predictions)
        print(predictions)
        print(y)
        print(sum(predictions==y)/test_count)
        return sum(predictions==y)/test_count
    
    def _predict(self, X_format, X_content):
        return self.predict_proba(X_format, X_content)
    
    def predict_proba(self, X_format, X_content):
        probas = []
        print("%%%%%%%%%%%%%%%%%%%%%%%%%%%")
        for name, clf in self.classifiers:
            print(name)
            if name == 'RandomForestClassifier':
                probas.append(clf.predict_proba(X_format))
                p1 = clf.predict(X_format)
                print(clf.predict_proba(X_format), clf.predict(X_format))
            else:
                probas.append(clf.predict_proba(X_content))
                p2 = clf.predict(X_content)
                print(clf.predict_proba(X_content), clf.predict(X_content))
        
        probas = np.asarray(probas)
        n = np.argmax(probas)
        
        print n

        if n in [0, 1]:
            return p1[0]
        return p2[0]
    
    def train(self, df):
        print('train *************************************')
        self.train_df = df
        print('process_data ******************************')
        self.process_data(df)
        print('triain_rfclassifier ***********************')
        self.triain_rfclassifier()
        print('triain_nbclassifier ***********************')
        self.triain_nbclassifier()
        print('predict ***********************************')
        print self.predict()
        
    def triain_rfclassifier(self):
        X, y = self.X_format, self.y
        clf = Pipeline([#('pca', PCA()),
                    ('clf', RandomForestClassifier()),
                ])

        parameters = {#'pca__n_components': (3, 4, 5, 6, 7),
#                         'clf__n_estimators': (5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 60, 70), 
#                         'clf__criterion': ('gini', 'entropy'),
#                         'clf__max_features': ('auto', 'sqrt', 'log2')
                     }    

        gs_clf = GridSearchCV(clf, parameters, cv=5)
        gs_clf.fit(X, y)

        best_parameters, score, _ = max(gs_clf.grid_scores_, key=lambda x: x[1])
        print('RandomForestClassifier')
        print('Score : ', score)
        for param_name in sorted(parameters.keys()):
            print("%s: %r" % (param_name, best_parameters[param_name]))
        
#         self.best_rfc = gs_clf.best_estimator_
        self.classifiers.append(('RandomForestClassifier', gs_clf.best_estimator_))
    
    def triain_nbclassifier(self):
        X, y = self.X_content, self.y
        clf = Pipeline([
                        ('clf', MultinomialNB()),
                    ])

        parameters = { 'clf__alpha': (0,1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3, 1.4), 
                        'clf__fit_prior':(True, False)}    

        gs_clf = GridSearchCV(clf, parameters, cv=5)
        gs_clf.fit(X, y)

        best_parameters, score, _ = max(gs_clf.grid_scores_, key=lambda x: x[1])
        print('MultinomialNB')
        print('Score : ', score)
        for param_name in sorted(parameters.keys()):
            print("%s: %r" % (param_name, best_parameters[param_name]))
        
#         self.best_nbc = gs_clf.best_estimator_
        self.classifiers.append(('MultinomialNB', gs_clf.best_estimator_))
        
    

In [170]:
mvc = MajorityVoteClassifier()
mvc.train(all_df)

train *************************************
process_data ******************************
triain_rfclassifier ***********************
RandomForestClassifier
('Score : ', 0.89023162134944611)
triain_nbclassifier ***********************
MultinomialNB
('Score : ', 0.90634441087613293)
clf__alpha: 0.3
clf__fit_prior: False
predict ***********************************
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 

C:\Users\uqbjeon\AppData\Local\Continuum\Miniconda2\envs\myenv\lib\site-packages\sklearn\utils\validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
C:\Users\uqbjeon\AppData\Local\Continuum\Miniconda2\envs\myenv\lib\site-packages\sklearn\utils\validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
C:\Users\uqbjeon\AppData\Local\Continuum\Miniconda2\envs\myenv\lib\site-packages\sklearn\utils\validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape

(array([[ 0.00770972,  0.99229028]]), array([1], dtype=int64))
1
%%%%%%%%%%%%%%%%%%%%%%%%%%%
RandomForestClassifier
(array([[ 0.,  1.]]), array([1], dtype=int64))
MultinomialNB
(array([[ 0.03688694,  0.96311306]]), array([1], dtype=int64))
1
%%%%%%%%%%%%%%%%%%%%%%%%%%%
RandomForestClassifier
(array([[ 0.,  1.]]), array([1], dtype=int64))
MultinomialNB
(array([[ 0.0148179,  0.9851821]]), array([1], dtype=int64))
1
%%%%%%%%%%%%%%%%%%%%%%%%%%%
RandomForestClassifier
(array([[ 0.,  1.]]), array([1], dtype=int64))
MultinomialNB
(array([[ 0.04666402,  0.95333598]]), array([1], dtype=int64))
1
%%%%%%%%%%%%%%%%%%%%%%%%%%%
RandomForestClassifier
(array([[ 0.,  1.]]), array([1], dtype=int64))
MultinomialNB
(array([[ 0.06173339,  0.93826661]]), array([1], dtype=int64))
1
%%%%%%%%%%%%%%%%%%%%%%%%%%%
RandomForestClassifier
(array([[ 0.,  1.]]), array([1], dtype=int64))
MultinomialNB
(array([[ 0.06491509,  0.93508491]]), array([1], dtype=int64))
1
%%%%%%%%%%%%%%%%%%%%%%%%%%%
RandomForestClassifier
(

C:\Users\uqbjeon\AppData\Local\Continuum\Miniconda2\envs\myenv\lib\site-packages\sklearn\utils\validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
C:\Users\uqbjeon\AppData\Local\Continuum\Miniconda2\envs\myenv\lib\site-packages\sklearn\utils\validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
C:\Users\uqbjeon\AppData\Local\Continuum\Miniconda2\envs\myenv\lib\site-packages\sklearn\utils\validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape

(array([[ 0.00876387,  0.99123613]]), array([1], dtype=int64))
3
%%%%%%%%%%%%%%%%%%%%%%%%%%%
RandomForestClassifier
(array([[ 0.,  1.]]), array([1], dtype=int64))
MultinomialNB
(array([[ 0.04559916,  0.95440084]]), array([1], dtype=int64))
1
%%%%%%%%%%%%%%%%%%%%%%%%%%%
RandomForestClassifier
(array([[ 0.1,  0.9]]), array([1], dtype=int64))
MultinomialNB
(array([[ 0.03977369,  0.96022631]]), array([1], dtype=int64))
3
%%%%%%%%%%%%%%%%%%%%%%%%%%%
RandomForestClassifier
(array([[ 0.,  1.]]), array([1], dtype=int64))
MultinomialNB
(array([[ 0.04797534,  0.95202466]]), array([1], dtype=int64))
1
%%%%%%%%%%%%%%%%%%%%%%%%%%%
RandomForestClassifier
(array([[ 0.,  1.]]), array([1], dtype=int64))
MultinomialNB
(array([[ 0.02833581,  0.97166419]]), array([1], dtype=int64))
1
%%%%%%%%%%%%%%%%%%%%%%%%%%%
RandomForestClassifier
(array([[ 0.1,  0.9]]), array([1], dtype=int64))
MultinomialNB
(array([[ 0.19868498,  0.80131502]]), array([1], dtype=int64))
1
%%%%%%%%%%%%%%%%%%%%%%%%%%%
RandomForestClassi

...

C:\Users\uqbjeon\AppData\Local\Continuum\Miniconda2\envs\myenv\lib\site-packages\sklearn\utils\validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
C:\Users\uqbjeon\AppData\Local\Continuum\Miniconda2\envs\myenv\lib\site-packages\sklearn\utils\validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
C:\Users\uqbjeon\AppData\Local\Continuum\Miniconda2\envs\myenv\lib\site-packages\sklearn\utils\validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape

(array([[ 0.03597495,  0.96402505]]), array([1], dtype=int64))
3
%%%%%%%%%%%%%%%%%%%%%%%%%%%
RandomForestClassifier
(array([[ 0.,  1.]]), array([1], dtype=int64))
MultinomialNB
(array([[ 0.06493574,  0.93506426]]), array([1], dtype=int64))
1
%%%%%%%%%%%%%%%%%%%%%%%%%%%
RandomForestClassifier
(array([[ 0.,  1.]]), array([1], dtype=int64))
MultinomialNB
(array([[ 0.05387925,  0.94612075]]), array([1], dtype=int64))
1
%%%%%%%%%%%%%%%%%%%%%%%%%%%
RandomForestClassifier
(array([[ 0.,  1.]]), array([1], dtype=int64))
MultinomialNB
(array([[ 0.02282447,  0.97717553]]), array([1], dtype=int64))
1
%%%%%%%%%%%%%%%%%%%%%%%%%%%
RandomForestClassifier
(array([[ 0.,  1.]]), array([1], dtype=int64))
MultinomialNB
(array([[ 0.01050781,  0.98949219]]), array([1], dtype=int64))
1
%%%%%%%%%%%%%%%%%%%%%%%%%%%
RandomForestClassifier
(array([[ 0.,  1.]]), array([1], dtype=int64))
MultinomialNB
(array([[ 0.02454561,  0.97545439]]), array([1], dtype=int64))
1
%%%%%%%%%%%%%%%%%%%%%%%%%%%
RandomForestClassifier

C:\Users\uqbjeon\AppData\Local\Continuum\Miniconda2\envs\myenv\lib\site-packages\sklearn\utils\validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
C:\Users\uqbjeon\AppData\Local\Continuum\Miniconda2\envs\myenv\lib\site-packages\sklearn\utils\validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
C:\Users\uqbjeon\AppData\Local\Continuum\Miniconda2\envs\myenv\lib\site-packages\sklearn\utils\validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape

(array([[ 0.,  1.]]), array([1], dtype=int64))
MultinomialNB
(array([[ 0.05693165,  0.94306835]]), array([1], dtype=int64))
1
%%%%%%%%%%%%%%%%%%%%%%%%%%%
RandomForestClassifier
(array([[ 0.,  1.]]), array([1], dtype=int64))
MultinomialNB
(array([[ 0.00337573,  0.99662427]]), array([1], dtype=int64))
1
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[0 1 0 1 0 1 0 1 0 0 0 0 0 1 0 0 0 0 0 0 1 1 1 0 1 1 0 1 1 0 0 1 0 0 1 0 0
 1 0 1 1 1 1 0 1 0 1 1 0 0 1 1 0 1 0 1 1 0 0 1 0 1 0 1 1 0 0 1 0 1 1 1 1 1
 0 1 0 0 1 0 1 1 1 1 1 0 0 0 1 1 0 1 1 1 0 0 0 0 0 0]
0.49
0.49


C:\Users\uqbjeon\AppData\Local\Continuum\Miniconda2\envs\myenv\lib\site-packages\sklearn\utils\validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
C:\Users\uqbjeon\AppData\Local\Continuum\Miniconda2\envs\myenv\lib\site-packages\sklearn\utils\validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
C:\Users\uqbjeon\AppData\Local\Continuum\Miniconda2\envs\myenv\lib\site-packages\sklearn\utils\validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape

## Read other articles & generate signals(features)

## Fetch signals(features) before applying algorithm

In [54]:
features = ['count_line', 'mean_word', 'total_word', 'std_w', 'mean_y', 'mean_h', 'mean_x', 'page_num', 'content']

In [55]:
other_x = others[features].values

In [56]:
other_x[other_x > 1000] = 1000
other_x[np.isnan(other_x)] = 0

In [57]:
poem_x = poems[features].values

In [58]:
X = np.concatenate((other_x, poem_x), axis=0)

In [59]:
other_y = others[['target']].values
poem_y = poems[['target']].values
y = np.concatenate((other_y, poem_y), axis=0)

In [62]:
y = y.flatten()
print X.shape, y.shape

(993L, 8L) (993L,)


In [63]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.cross_validation import cross_val_score, KFold
from sklearn import decomposition
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn.decomposition import TruncatedSVD
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.grid_search import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA

## Applying different algoriths with only newspaper format signal to check  accurcy

### 1. Applying SVC Classifier(Support Vector Machine)

In [64]:
svc = SGDClassifier()
svc.fit(X, y)
scores = cross_val_score(svc, X, y, cv=5)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.58 (+/- 0.13)


### 2. Appliying Random Forest Classifier

In [65]:
forest = RandomForestClassifier()
forest.fit(X, y)
scores = cross_val_score(forest, X, y, cv=5)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.91 (+/- 0.01)


### 3. Appliying Logistic Regression Classifier

In [66]:
logistic = LogisticRegression()
logistic.fit(X, y)
scores = cross_val_score(logistic, X, y, cv=5)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.80 (+/- 0.06)


### 4. Appliying Naive Bayes Classifier

In [67]:
nb = MultinomialNB()
nb.fit(X, y)
scores = cross_val_score(nb, X, y, cv=5)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.64 (+/- 0.09)


## Find optimization in Random Forest Classifier which is the best classifier

In [68]:
clf = Pipeline([#('pca', PCA()),
                    ('clf', RandomForestClassifier()),
                ])

parameters = {#'pca__n_components': (3, 4, 5, 6, 7),
                'clf__n_estimators': (5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 60, 70), 
                'clf__criterion': ('gini', 'entropy'),
                'clf__max_features': ('auto', 'sqrt', 'log2')
             }    

gs_clf = GridSearchCV(clf, parameters, cv=5)
gs_clf.fit(X, y)

best_parameters, score, _ = max(gs_clf.grid_scores_, key=lambda x: x[1])
print('Score : ', score)
for param_name in sorted(parameters.keys()):
    print("%s: %r" % (param_name, best_parameters[param_name]))


('Score : ', 0.93353474320241692)
clf__criterion: 'gini'
clf__max_features: 'auto'
clf__n_estimators: 70


## Find important singals  

In [69]:
clf = RandomForestClassifier(n_estimators=50, max_features='sqrt', criterion='gini')
clf.fit(X, y)

print sorted(zip(map(lambda x: round(x, 4), clf.feature_importances_), features), reverse=True)

[(0.2245, 'page_num'), (0.1505, 'mean_y'), (0.1357, 'mean_h'), (0.1252, 'mean_word'), (0.1182, 'total_word'), (0.0954, 'count_line'), (0.0934, 'mean_x'), (0.0573, 'std_w')]


#### Best signals

1. mean word per line
1. newspaper page number
1. indent x

### Apply Naive Bayes algorithm on poem and other articles

In [69]:
X = np.concatenate((poems['content'].values, others['content'].values), axis=0)
print type(X)
print X.shape
vectorizer = CountVectorizer(max_df=10, min_df=1)
X = vectorizer.fit_transform(X)
X = TfidfTransformer().fit_transform(X)
  

<type 'numpy.ndarray'>
(993L,)


In [70]:
X = all_df['content'].values
print type(X)
print X.shape
vectorizer = CountVectorizer(max_df=10, min_df=1)
X = vectorizer.fit_transform(X)
X = TfidfTransformer().fit_transform(X)

<type 'numpy.ndarray'>
(993L,)


In [71]:
nb = MultinomialNB()
nb.fit(X, y)

scores = cross_val_score(nb, X, y, cv=5)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.91 (+/- 0.03)


In [76]:
clf = Pipeline([
#                 ('vect', CountVectorizer()),
#                 ('tfidf', TfidfTransformer()),
                ('clf', MultinomialNB()),
            ])
    
parameters = {
#                 'tfidf__use_idf': (True, False),
                'clf__alpha': (0,1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3, 1.4), 
                'clf__fit_prior':(True, False)}    
    
    
gs_clf = GridSearchCV(clf, parameters, cv=5)

gs_clf.fit(X, y)

best_parameters, score, _ = max(gs_clf.grid_scores_, key=lambda x: x[1])
print('Score : ', score)
for param_name in sorted(parameters.keys()):
    print("%s: %r" % (param_name, best_parameters[param_name]))

('Score : ', 0.91339375629405839)
clf__alpha: 1
clf__fit_prior: True


### Combine the two best classifiers(RandomForest classifier on newspaper format and Naive Bayes on Content)

In [78]:
class MajorityVoteClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, classifiers, weights=None):
        self.classifiers = classifiers
        self.named_classifiers = {key: value for key, value in _name_estimators(classifiers)}
        print(self.named_classifiers)
        self.weights = weights
    
    def fit(self, X, y):
        self.classifiers_ = []
        for clf in self.classifiers:
            
            fitted_clf = clone(clf).fit(X, y)
            self.classifiers_.append(fitted_clf)
        return self

    def predict(self, X):
        return np.argmax(self.predict_proba(X), axis=1)
    
    def predict_proba(self, X):
        probas = np.asarray([cls.predict_proba(X) for clf in self.classifiers_])
        avg_proba = np.average(probas, axis=0, weights=self.weights)
        return avg_proba
    
    def get_params(self, deep=True):
        if not deep:
            return super(MajorityVoteClassifier, self).get_params(deep=False)
        else:
            out = self.named_classifiers.copy()
            for name, step in six.iteritems(self.named_classifiers):
                for key, value in six.iteritems(step.get_params(deep=True)):
                    out['%s__%s' % (name, key)] = value
            return out

In [135]:
a = [[100, 12]]
b = [[3, 4]]
c = []
c.append(a)
c.append(b)
c = np.array(c)
np.argmax(c)

0